In [91]:
import numpy as np
import random
import math
import copy
from sortedcontainers import SortedList
from IPython.display import clear_output

from util import read_nxgraph
from util import obj_maxcut

In [92]:
class Bucket:
    def __init__(self, vertexList, graph):
        self.graph = graph
        self.solution = vertexList
    
        self.vertexList = {}

        for i in range(len(vertexList)):
            self.vertexList[i] = vertexList[i]

        self.buckets0 = {}
        self.buckets1 = {}

        self.gains0 = SortedList()
        self.gains1 = SortedList()

        self.maxgain0 = None
        self.maxgain1 = None

        self.neighbors = {}

        for i in range(graph.number_of_nodes()):
            self.neighbors[i] = set(graph.neighbors(i))
            
    def calcGain(self, vertex):
        # Save the original value of the vertex
        original_value = self.solution[vertex]
        # Flip the vertex value in the current solution
        self.solution[vertex] = 1 if original_value == 0 else 0

        gain = 0
        # Iterate over neighbors of the vertex
        for neighbor in self.graph.neighbors(vertex):
            edge_weight = self.graph[vertex][neighbor].get('weight', 1)  # Default weight is 1 if not provided
            neighbor_value = self.solution[neighbor]
            
            if original_value == neighbor_value:  # Originally on the same side of the cut
                gain += int(edge_weight)
            else:  # Originally on opposite sides of the cut
                gain -= int(edge_weight)

        # Restore the original value
        self.solution[vertex] = original_value

        return gain

    def addVertex(self, vertex, partition):
        gain = self.calcGain(vertex) # Get gain value from vertex i
        bucketDict = self.buckets1 if partition == 1 else self.buckets0

        if gain not in bucketDict:
            bucketDict[gain] = {}
            bucketDict[gain][vertex] = partition
            gainsList = self.gains0 if partition == 0 else self.gains1
            gainsList.add(gain)
            if partition == 0:
                self.gains0 = gainsList
            else:
                self.gains1 = gainsList
        else:
            bucketDict[gain][vertex] = partition
            
        self.vertexList[vertex] = (partition, gain)

        pointer = self.maxgain1 if partition == 1 else self.maxgain0
        pointer = gain if pointer == None or gain > pointer else pointer
        if partition == 1:
            self.maxgain1 = pointer
        else:
            self.maxgain0 = pointer

    # Remove vertex from its bucket
    def removeVertex(self, vertex):
        if vertex in self.vertexList:
            partition, gain = self.vertexList[vertex]
            bucketDict = self.buckets1 if partition == 1 else self.buckets0

            if gain in bucketDict:
                del bucketDict[gain][vertex]
            else:
                print("Gain not present in Bucket")

            if not bucketDict[gain]:    
                gainsList = self.gains0 if partition == 0 else self.gains1
                gainsList.remove(gain)
            
            if not bucketDict[gain]:
                pointer = self.maxgain1 if partition == 1 else self.maxgain0
                if(len(gainsList) >= 1):
                    pointer = gainsList[-1] if gain == pointer else pointer
                if partition == 1:
                    self.maxgain1 = pointer
                else:
                    self.maxgain0 = pointer
                del bucketDict[gain]
            del self.vertexList[vertex]
 

    # Move vertex from one partition to the other
    def moveVertex(self, vertex):
        if vertex in self.vertexList:
            partition, _ = self.vertexList[vertex]
            self.removeVertex(vertex)
            self.addVertex(vertex, partition=1) if partition == 0 else self.addVertex(vertex, partition=0)
            self.solution[vertex] = 0 if self.solution[vertex] == 1 else 1
        
        for i in self.neighbors[vertex]:
            if i in self.vertexList:
                partition, _ = self.vertexList[i]
                self.removeVertex(i)
                self.addVertex(i, partition=1) if partition == 0 else self.addVertex(i, partition=0)
                self.solution[i] = 0 if self.solution[i] == 1 else 1

    def xorOP(self, vertex1, vertex2):
        returningBucket = copy.deepcopy(self)
        returningBucket.moveVertex(vertex1) if vertex1 != None else None 
        returningBucket.moveVertex(vertex2) if vertex2 != None else None

        return returningBucket

    def get_best_move(self):
        bestA = None if self.maxgain0 is None else self.buckets0[self.maxgain0][next(iter(self.buckets0[self.maxgain0]))]
        bestB = None if self.maxgain1 is None else self.buckets1[self.maxgain1][next(iter(self.buckets1[self.maxgain1]))]

        return int(bestA) if self.maxgain0 >= self.maxgain1 else int(bestB)
    
    def getBestMoveA(self):
        bestA = None if self.maxgain0 is None else self.buckets0[self.maxgain0][next(iter(self.buckets0[self.maxgain0]))]

        return int(bestA)
    
    def getBestMoveB(self):
        bestB = None if self.maxgain1 is None else self.buckets1[self.maxgain1][next(iter(self.buckets1[self.maxgain1]))]

        return int(bestB)
    
    def get_combined_buckets(self):
        return self.solution


In [93]:
graph = read_nxgraph('.././data/gset/gset_14.txt')
currSolution = list(np.random.randint(0, 2, graph.number_of_nodes()))
bucket = Bucket(currSolution, graph)
for i in range(graph.number_of_nodes()):
    if(currSolution[i] == 0):
        bucket.addVertex(i, 0)
    else:
        bucket.addVertex(i, 1)

currSolution = bucket
currVal = obj_maxcut(currSolution.get_combined_buckets(), graph)
bestSolution = currSolution
bestVal = currVal
prevSolution = currSolution

In [94]:
omega = 0
iteration = 0
L0 = int(0.01*graph.number_of_nodes())
L = L0
T = 1000
gamma = random.randint(3,(graph.number_of_nodes()/10))
P0 = 0.8
Q = 0.5
H = {}

In [95]:
def isNotTabu(H, vertex, gamma, iteration):
    if vertex in H:
        if((H[vertex]+gamma) < iteration):
            return True
    return False

In [96]:
def perturb(C,L,M,iteration,H):
    for _ in range(L):
        bestVertex = C.get_best_move() 
        bestVertexA = C.getBestMoveA() 
        bestVertexB = C.getBestMoveB() 
        if(M == 'A2' and (isNotTabu(H, bestVertexA, gamma, iteration) and isNotTabu(H, bestVertexB, gamma, iteration))):
            combo = C.xorOP(bestVertexA, bestVertexB)
            H[bestVertexA] = iteration + gamma
            H[bestVertexB] = iteration + gamma
        elif(M == 'A1' and isNotTabu(H, bestVertex, gamma, iteration)):
            combo = C.xorOP(bestVertex, None)
            H[bestVertex] = iteration + gamma
        else:
            randomVertex = random.randint(0, len(C.get_combined_buckets())-2)
            combo = C.xorOP(randomVertex, None)
            H[randomVertex] = iteration + gamma
        
        C = combo
    iteration = iteration + 1
    
    return C, H, iteration

In [97]:
def perturbation(C, L, H, iteration, omega):
    if omega == 0:
        C, H, iteration = perturb(C,L,'B',iteration,H)
    else:
        if (np.exp(-1*(omega)/T) > P0):
            P = np.exp(-1*(omega)/T)
        else:
            P = P0

        randProb = random.random()

        if (randProb < (P * Q)):
            C, H, iteration = perturb(C,L,'A1',iteration,H)
        elif (randProb < (P * Q + P * (1-Q))):
            C, H, iteration = perturb(C,L,'A2',iteration,H)
        else:
            C, H, iteration = perturb(C,L,'B',iteration,H)

    return C, H, iteration

In [98]:
while(iteration < 200000*graph.number_of_nodes()):
    vertexToMove  = currSolution.get_best_move()
    combo = currSolution.xorOP(vertexToMove, None)

    while(obj_maxcut(combo.get_combined_buckets(), graph) > currVal):
        currVal = obj_maxcut(combo.get_combined_buckets(), graph)
        currSolution = combo
        vertexToMove = currSolution.get_best_move()
        if isNotTabu(H, vertexToMove, gamma, iteration):
            combo = currSolution.xorOP(vertexToMove, None)
            newVal = currVal + currSolution.newGain(vertexToMove, None)
            H[vertexToMove] = iteration + gamma 
            iteration = iteration + 1
    
    if currVal > bestVal:
        bestSolution = currSolution
        bestVal = currVal
        omega = 0
    else:
        omega = omega + 1

    # Determine the number of perturbation moves L to be applied to C

    if omega > T:
        # Search is stagnating, random perturbation is required
        omega = 0

    if currSolution == prevSolution:
        L = int(L + 1)
    else:
        L = int(L0)

    prevSolution = currSolution
    currSolution, H, iteration = perturbation(currSolution, L, H, iteration, omega)

    clear_output(wait=True)
    print('bestVal:', bestVal, 'at iteration:', iteration)

bestVal: 2449 at iteration: 148


KeyboardInterrupt: 

In [ ]:
print(currVal)
print(obj_maxcut(bestSolution.get_combined_buckets(), graph))

3699
2300
